In [ ]:
#!pip install pyreadr
#!pip install imbalanced-learn
import pyreadr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import IsolationForest

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
def encode_variables(df):
  '''Using one-hot encoding to divide categorical features to binary
  splitting multi-categorical values over several columns.'''
  
  #rename labels into english
  df.fd.replace('ПФО', 'PFO', inplace=True)

  #Generate column and lables with one hot encoding
  df = pd.get_dummies(df, columns = ['fd'], prefix='fd') 

  return df

In [ ]:
def handle_missing_values(df, row_thresh=None, col_thresh=None):
  ''' Removes rows and columns from given dataframe object with a number of 
  missing values above the given threshholds. Removes rows with any nan if threshold arguemnt is not given'''

  #Remove rows with more than row_thresh NaN Values  
  if (row_thresh != None):  
    thresh_count_row = int((1-row_thresh) * df.shape[1] + 1)
    df = df.dropna(axis=0, thresh=thresh_count_row).copy()

  #Remove columns with more than col_thresh NaN Values    
  if (col_thresh != None):
    thresh_count_col = int((1-col_thresh) * df.shape[0] + 1)
    df = df.dropna(axis=1, thresh=thresh_count_col).copy()

  #Remove rows with any missing values
  if (row_thresh == None) and (col_thresh == None):
    df = df.dropna()

  return df

In [ ]:
def remove_outliers(df, contamination=0.1):
  ''' Gives indexes of outliers detected by sklearn.IsolationForest() in given dataset. '''

  #identifying outliers
  clf = IsolationForest(contamination=contamination).fit(df)
  outliers = clf.predict(df)

  #returns the position of outliers in the dataframe
  outlier_idx = outliers != -1

  return outlier_idx

In [ ]:
def filter_data(df):
  '''Removing specified columns that are considered irrelevant for the a-priori
  classification task and removes filters that do not match pre-defined values. '''

  #Filtering rows based on variable values 
  df = df[df['platform'] == 'Sber'].reset_index(drop=True)
  df = df[df['pmeth'] == 'EA'].reset_index(drop=True)
  df = df[df['law'] == '44-FZ'].reset_index(drop=True)
  df = df[df['nAppr1sber'] >= 2].reset_index(drop=True)

  #Removing irrelevant columns for classification task
  remove_cols = ['row', 'aucID', 'nAppr1gos','law', 'nRej1gos', 'pathol', 'innORG', 'innBUY', 'icpReducPct', 'platform', 'pmeth', 'icpOR', 'aucYearOR', 'firstBidDur', 'bv1hour', 'contractSec']
  df.drop(remove_cols, inplace=True, axis=1)

  # filtering only prior features
  df_prior = df.copy()
  remove_post = ['wpp', 'nAppr2', 'nPart', 'single', 'invalid', 'broke'] # remove posterior features
  df_prior.drop(remove_post, inplace=True, axis=1)
  filtered_shape = df_prior.shape

  return df_prior

In [ ]:
def scale_variables(df):
  ''' Scaling non-binary numerical variables to scale of 0 to 1. '''

  scaling_list = ["icpKZ", "nAppr1sber", "claimSec", "n1hour"]

  scaler = RobustScaler()
  robust_df = scaler.fit_transform(df)
  robust_df = pd.DataFrame(robust_df, index=df.index, columns=df.columns)

  norm_scaler = MinMaxScaler()
  normalized = norm_scaler.fit_transform(robust_df)
  df = pd.DataFrame(normalized, index=df.index, columns=df.columns)


  #normalization by min/max
  #for column in scaling_list:
  #df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())   

  #if (scale_type == 'stan'):
  #  scaler = StandardScaler()
  #  standardized = scaler.fit_transform(df.values)
  #  df = pd.DataFrame(standardized, index=df.index, columns=df.columns)

  #for column in scaling_list:
  #  df[column] = np.log(df[column])   

  return df

In [ ]:
def oversample(X, y):
  '''Performes oversampling/upsampling of training data and corresponding labels 
  using imblearn.SMOTE().'''
  
  counter = Counter(y)
  print(counter)

  # transform the dataset
  oversample = SMOTE()
  X, y = oversample.fit_resample(X, y)
  
  # summarize the new class distribution
  counter = Counter(y)
  print(counter)
  
  return X, y

In [ ]:
def create_splits(df, labelcol):
  ''' Creates train and test splits with labels from dataframe using sklearn 
  built-in-function. Takes dataframe and name of column with labels.'''

  x = df.loc[:, df.columns != labelcol]
  y = df.collusion
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=6)

  print('Shape of x and y before split', x.shape, y.shape)
  print('Shape of x_train and x_test after split', x_train.shape, x_test.shape)
  print('Train set contains {0} pct ratio of collusion labels'.format((y_train.sum()/y_train.shape[0])*100))
  print('Test set contains {0} pct ratio of collusion labels\n'.format((y_test.sum()*100/y_test.shape[0])))

  return x_train, x_test, y_train, y_test

In [ ]:
 #Connnect to google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Uncomment to run

In [ ]:
# #loading hiv and tubercolosis data
# data_hiv = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataHIV.Rdata')['dfHIV']
# data_tb = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataTB.RData')['dfTB']
# data_combined = pd.concat([data_hiv, data_tb], ignore_index=True)
# df_comb = data_combined.copy()

# df_comb = df_comb.drop('aucDateOR', axis=1)

# #imputing nan value for ClaimSec
# df_comb.loc[df_comb[df_comb.aucID == "0306500000117000001"].index,"claimSec"] = 1611.80
# df_comb.loc[df_comb[df_comb.aucID == "0356200006718000097"].index,"claimSec"] = 424.1

# #imputing nan value for bvSameBO
# df_comb.loc[df_comb[df_comb.aucID == "0888500000219000457"].index,"bvSameBO"] = 1

In [ ]:
# df_comb = encode_variables(df_comb)
# print('Before filtering', df_comb.shape)

#df_comb = filter_data(df_comb)
#df_comb = scale_variables(df_comb)

#print('Before handling missing values', df_comb.shape)
#df_comb_clean = handle_missing_values(df_comb.copy(), 0.25, 0.9)

#print('After preprocessing', df_comb.shape)

#split on cleaned data
#x_train, x_test, y_train, y_test = create_splits(df_comb_clean, 'collusion')


#split on data with removed nans
#df_comb_removed = handle_missing_values(df_comb)
#print('After handling missing values', df_comb_removed.shape)

#df_comb_removed = df_comb.dropna().copy()
#x_train_r, x_test_r, y_train_r, y_test_r = create_splits(df_comb_removed, 'collusion')

#mask = remove_outliers(x_train_r, 0.15)
#x_train_r, y_train_r = x_train_r.iloc[mask, :], y_train_r.iloc[mask] 
#print('After handling outliers', x_train_r.shape)

#x_train_r, y_train_r = oversample(x_train_r, y_train_r)

Before filtering (187287, 45)


In [ ]:

df_comb = filter_data(df_comb)
df_comb = scale_variables(df_comb)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:1565: RuntimeWarning: invalid value encountered in subtract
  X -= self.center_


In [ ]:
data_hiv = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataHIV.Rdata')['dfHIV']
data_tb = pyreadr.read_r('/content/gdrive/MyDrive/ML-exam/dataprocessing/dataTB.RData')['dfTB']

In [ ]:
data_tb.shape

(148720, 39)